# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 8 2022 12:38PM,241424,10,21156852,"ClearSpec, LLC",Released
1,Jul 8 2022 12:35PM,241423,19,ADV80003993,"AdvaGen Pharma, Ltd",Released
2,Jul 8 2022 12:35PM,241423,19,ADV80003994,"AdvaGen Pharma, Ltd",Released
3,Jul 8 2022 12:35PM,241423,19,ADV80003995,"AdvaGen Pharma, Ltd",Released
4,Jul 8 2022 12:35PM,241423,19,ADV80003996,"AdvaGen Pharma, Ltd",Released
5,Jul 8 2022 12:35PM,241423,19,ADV80003997,"AdvaGen Pharma, Ltd",Released
6,Jul 8 2022 12:34PM,241421,102,PBR-062422-05,"Senseonics, Incorporated",Released
7,Jul 8 2022 12:29PM,241419,19,ADV80003977,"AdvaGen Pharma, Ltd",Released
8,Jul 8 2022 12:29PM,241419,19,ADV80003968,"AdvaGen Pharma, Ltd",Released
9,Jul 8 2022 12:29PM,241419,19,ADV80003972,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,241419,Released,11
47,241420,Released,1
48,241421,Released,1
49,241423,Released,5
50,241424,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241419,NaN,NaN,11.0
241420,NaN,NaN,1.0
241421,NaN,NaN,1.0
241423,NaN,NaN,5.0
241424,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241343,0.0,1.0,0.0
241345,0.0,1.0,0.0
241347,0.0,0.0,1.0
241348,0.0,0.0,1.0
241351,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241343,0,1,0
241345,0,1,0
241347,0,0,1
241348,0,0,1
241351,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241343,0,1,0
1,241345,0,1,0
2,241347,0,0,1
3,241348,0,0,1
4,241351,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241343,,1,
1,241345,,1,
2,241347,,,1
3,241348,,,1
4,241351,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 8 2022 12:38PM,241424,10,"ClearSpec, LLC"
1,Jul 8 2022 12:35PM,241423,19,"AdvaGen Pharma, Ltd"
6,Jul 8 2022 12:34PM,241421,102,"Senseonics, Incorporated"
7,Jul 8 2022 12:29PM,241419,19,"AdvaGen Pharma, Ltd"
18,Jul 8 2022 12:27PM,241420,10,"ClearSpec, LLC"
19,Jul 8 2022 12:06PM,241414,10,"ClearSpec, LLC"
20,Jul 8 2022 12:06PM,241413,19,ACG North America LLC
22,Jul 8 2022 12:03PM,241412,10,"Methapharm, Inc."
40,Jul 8 2022 11:48AM,241409,10,ISDIN Corporation
64,Jul 8 2022 11:44AM,241408,10,Livs Products


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 8 2022 12:38PM,241424,10,"ClearSpec, LLC",,,1
1,Jul 8 2022 12:35PM,241423,19,"AdvaGen Pharma, Ltd",,,5
2,Jul 8 2022 12:34PM,241421,102,"Senseonics, Incorporated",,,1
3,Jul 8 2022 12:29PM,241419,19,"AdvaGen Pharma, Ltd",,,11
4,Jul 8 2022 12:27PM,241420,10,"ClearSpec, LLC",,,1
5,Jul 8 2022 12:06PM,241414,10,"ClearSpec, LLC",,,1
6,Jul 8 2022 12:06PM,241413,19,ACG North America LLC,,,2
7,Jul 8 2022 12:03PM,241412,10,"Methapharm, Inc.",,,18
8,Jul 8 2022 11:48AM,241409,10,ISDIN Corporation,,,24
9,Jul 8 2022 11:44AM,241408,10,Livs Products,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 12:38PM,241424,10,"ClearSpec, LLC",1,,
1,Jul 8 2022 12:35PM,241423,19,"AdvaGen Pharma, Ltd",5,,
2,Jul 8 2022 12:34PM,241421,102,"Senseonics, Incorporated",1,,
3,Jul 8 2022 12:29PM,241419,19,"AdvaGen Pharma, Ltd",11,,
4,Jul 8 2022 12:27PM,241420,10,"ClearSpec, LLC",1,,
5,Jul 8 2022 12:06PM,241414,10,"ClearSpec, LLC",1,,
6,Jul 8 2022 12:06PM,241413,19,ACG North America LLC,2,,
7,Jul 8 2022 12:03PM,241412,10,"Methapharm, Inc.",18,,
8,Jul 8 2022 11:48AM,241409,10,ISDIN Corporation,24,,
9,Jul 8 2022 11:44AM,241408,10,Livs Products,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 12:38PM,241424,10,"ClearSpec, LLC",1,,
1,Jul 8 2022 12:35PM,241423,19,"AdvaGen Pharma, Ltd",5,,
2,Jul 8 2022 12:34PM,241421,102,"Senseonics, Incorporated",1,,
3,Jul 8 2022 12:29PM,241419,19,"AdvaGen Pharma, Ltd",11,,
4,Jul 8 2022 12:27PM,241420,10,"ClearSpec, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 12:38PM,241424,10,"ClearSpec, LLC",1.0,NaN,NaN
1,Jul 8 2022 12:35PM,241423,19,"AdvaGen Pharma, Ltd",5.0,NaN,NaN
2,Jul 8 2022 12:34PM,241421,102,"Senseonics, Incorporated",1.0,NaN,NaN
3,Jul 8 2022 12:29PM,241419,19,"AdvaGen Pharma, Ltd",11.0,NaN,NaN
4,Jul 8 2022 12:27PM,241420,10,"ClearSpec, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 12:38PM,241424,10,"ClearSpec, LLC",1.0,0.0,0.0
1,Jul 8 2022 12:35PM,241423,19,"AdvaGen Pharma, Ltd",5.0,0.0,0.0
2,Jul 8 2022 12:34PM,241421,102,"Senseonics, Incorporated",1.0,0.0,0.0
3,Jul 8 2022 12:29PM,241419,19,"AdvaGen Pharma, Ltd",11.0,0.0,0.0
4,Jul 8 2022 12:27PM,241420,10,"ClearSpec, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6517543,168.0,2.0,1.0
102,241421,1.0,0.0,0.0
12,241380,3.0,7.0,0.0
15,241378,136.0,24.0,8.0
16,241373,0.0,1.0,0.0
18,241351,0.0,1.0,0.0
19,965621,18.0,4.0,21.0
20,724163,20.0,8.0,1.0
21,965383,2.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6517543,168.0,2.0,1.0
1,102,241421,1.0,0.0,0.0
2,12,241380,3.0,7.0,0.0
3,15,241378,136.0,24.0,8.0
4,16,241373,0.0,1.0,0.0
5,18,241351,0.0,1.0,0.0
6,19,965621,18.0,4.0,21.0
7,20,724163,20.0,8.0,1.0
8,21,965383,2.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,168.0,2.0,1.0
1,102,1.0,0.0,0.0
2,12,3.0,7.0,0.0
3,15,136.0,24.0,8.0
4,16,0.0,1.0,0.0
5,18,0.0,1.0,0.0
6,19,18.0,4.0,21.0
7,20,20.0,8.0,1.0
8,21,2.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,168.0
1,102,Released,1.0
2,12,Released,3.0
3,15,Released,136.0
4,16,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,21
Status,,,,,,,,,
Completed,1.0,0.0,0.0,8.0,0.0,0.0,21.0,1.0,0.0
Executing,2.0,0.0,7.0,24.0,1.0,1.0,4.0,8.0,2.0
Released,168.0,1.0,3.0,136.0,0.0,0.0,18.0,20.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,21
0,Completed,1.0,0.0,0.0,8.0,0.0,0.0,21.0,1.0,0.0
1,Executing,2.0,0.0,7.0,24.0,1.0,1.0,4.0,8.0,2.0
2,Released,168.0,1.0,3.0,136.0,0.0,0.0,18.0,20.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,21
0,Completed,1.0,0.0,0.0,8.0,0.0,0.0,21.0,1.0,0.0
1,Executing,2.0,0.0,7.0,24.0,1.0,1.0,4.0,8.0,2.0
2,Released,168.0,1.0,3.0,136.0,0.0,0.0,18.0,20.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()